In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from neo4j import GraphDatabase

c:\Users\aalperen.arda\AppData\Local\anaconda3\envs\llm_doc_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [3]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [4]:
class Neo4jClient:
    def __init__(self, uri, user, password, db="database"):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.db = db

    def close(self):
        self.driver.close()

    def run_query(self, cypher_query):
        with self.driver.session(database=self.db) as session:
            result = session.run(cypher_query)
            return [record.data() for record in result]

In [5]:
query_generator_prompt = """
You are an AI assistant that only converts Turkish user requests into valid Cypher queries for querying a Neo4j graph database.

If the user request is conversational (e.g., greetings, confirmations, acknowledgements like “tamam”, “teşekkürler”, “selam”, etc.), do NOT return a query.
Instead, respond naturally in Turkish, like:
- “Rica ederim, başka bir isteğiniz olursa yardımcı olmaktan memnuniyet duyarım.”
- “Merhaba! Size nasıl yardımcı olabilirim?”
- “Tamamdır, bekliyorum 🙂”

If the user request is a query about people, connections, or database lookup — only then generate a valid Cypher query.

DO NOT include explanations for the Cypher queries. Just the query.

The database contains nodes of type `Person` with the following properties:
{
    "ad": "Ahmet Doğan",
    "dogum_yeri": "Kayseri",
    "yas": 37,
    "ilkokul": "Hacı Bektaş-ı Veli İlkokulu",
    "ilkokul_yillari": "1994-2002",
    "lise": "Ankara Fen Lisesi",
    "lise_yillari": "2002-2006",
    "universite": "Orta Doğu Teknik Üniversitesi (ODTÜ)",
    "universite_yillari": "2006-2010",
    "bolum": "Bilgisayar Mühendisliği",
    "yuksek_lisans": "Koç Üniversitesi",
    "yuksek_lisans_yillari": "2010-2012",
    "doktora": null,
    "doktora_yillari": null,
    "calistigi_kurumlar": ["ASELSAN", "SAP Türkiye", "Artemis Yazılım"],
    "calisma_yillari": ["2010-2014", "2014-2017", "2017-devam ediyor"],
    "yasadigi_sehir": "İstanbul",
    "hobiler": ["Tarihi roman okumak", "amatör drone fotoğrafçılığı", "bağlama çalmak", "açık kaynak yazılım projelerine katkıda bulunmak"],
    "es": "Elif Doğan",
    "cocuklar": ["Deniz Doğan", "Ada Doğan"],
    "dosya_adi": "ahmet_dogan.md"
}

Relationships between people use the `RELATED` relationship type.
The meaning of each connection is stored in a `type` property (e.g., 'UNIVERSITE_ARKADASI', 'AYNI_MEMLEKETTEN').

Examples:
User: Ben Gülnur Yıldız, ASELSAN’da çalışan Ahmet Doğan adlı kişiye nasıl ulaşabilirim?
Cypher query: MATCH p=(kaynak:Person {ad: 'Gülnur Yıldız'})-[r:RELATED*1..3]->(hedef:Person {ad: 'Ahmet Doğan'}) WHERE 'ASELSAN' IN hedef.calistigi_kurumlar RETURN p LIMIT 1

User: Teşekkür ederim.
Response: Rica ederim, başka bir isteğiniz olursa yardımcı olmaktan memnuniyet duyarım.

User: Merhaba
Response: Merhaba! Size nasıl yardımcı olabilirim?

User: Ahmet Doğan ile aynı üniversitede okuyan kişileri göster.
Cypher query: MATCH (p1:Person {ad: 'Ahmet Doğan'})<-[:RELATED]-(p2:Person) WHERE p1.universite = p2.universite RETURN DISTINCT p2
"""





result_interpreter_prompt = ""
result_interpreter_prompt += "You are a helpful assistant that receives structured data (such as Neo4j query results) and explains the findings in natural, fluent Turkish."

result_interpreter_prompt += " Your input will be a Neo4j response in the form of paths, nodes, or JSON structure that represent how people are connected."

result_interpreter_prompt += " Based on the graph data, summarize the path in Turkish and explain how the source person can reach the target person. Mention mutual schools, work places, or hometown connections if relevant."

result_interpreter_prompt += " Be concise, natural, and helpful. Do not invent or hallucinate any information. Base your answer strictly on the graph data."

result_interpreter_prompt += " Example input: A graph showing Gülnur Yıldız → Gül Yıldırım (üniversite arkadaşı) → Ali Türkmen (hemşehrisi) → Ahmet Doğan (lise arkadaşı, ASELSAN çalışanı)."
result_interpreter_prompt += " Output: Elimdeki bilgilere göre Gül Yıldırım'ın üniversite arkadaşı olduğunu görüyorum. Gül Yıldırım'ın hemşehrisi Ali Türkmen var. Ali Türkmen'in lise arkadaşı ise Ahmet Doğan ve şu anda ASELSAN'da çalışıyor."

result_interpreter_prompt += " Important: Your output must always be in Turkish."



In [6]:
def chat_1(message):
    messages = [
        {"role": "system", "content": query_generator_prompt},
        {"role": "user", "content": message}
      ]
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    query = completion.choices[0].message.content
    return query

In [7]:
'''cypher_query = chat_1("Ben Gülnur Yıldız, Ali Türkmen adlı kişiye nasıl ulaşabilirim?")
neo4j_client = Neo4jClient(
    uri="bolt://localhost:7687", 
    user="neo4j", 
    password="alperen2239",
    db="database"
)

result = neo4j_client.run_query(cypher_query)
print(result)'''

'cypher_query = chat_1("Ben Gülnur Yıldız, Ali Türkmen adlı kişiye nasıl ulaşabilirim?")\nneo4j_client = Neo4jClient(\n    uri="bolt://localhost:7687", \n    user="neo4j", \n    password="alperen2239",\n    db="database"\n)\n\nresult = neo4j_client.run_query(cypher_query)\nprint(result)'

In [8]:
def chat_2(history, user_message):
    messages = [{"role": "system", "content": result_interpreter_prompt}] + history + [{"role": "user", "content": user_message}]

    completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
    )
    return completion.choices[0].message.content

In [9]:
def chat_pipeline(message, history):
    cypher_query = chat_1(message)

    if cypher_query.lower().startswith("cypher query:"):
        cypher_query = cypher_query.split(":", 1)[1].strip()

    neo4j_client = Neo4jClient(
        uri="bolt://localhost:7687",
        user="neo4j",
        password="alperen2239",
        db="database"
    )

    result = neo4j_client.run_query(cypher_query)
    neo4j_client.close()

    user_message = f"Original question: {message}\nQuery Result: {result}\nBased on this result, explain how the people are connected. Your explanation must be in Turkish."

    response = chat_2(history, user_message)

    return chat_2(history, response)


In [10]:
gr.ChatInterface(fn=chat_pipeline, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: yili)} {position: line: 1, column: 165, offset: 164} for query: "MATCH (ben:Person {ad: 'Ali Türkmen'})-[r:RELATED]->(diger:Person) WHERE 'ASELSAN' IN diger.calistigi_kurumlar RETURN diger.yasadigi_sehir, duration.between(date(r.yili), date()).years AS calisma_yili"
